<a href="https://colab.research.google.com/github/ashraful-iut/NLP_Basic-/blob/main/Template_Pytorch_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import os
import glob
import pandas as pd
import random
from math import ceil
from time import time
import io
import json
import numpy as np
import matplotlib.pylab as plt
import shutil
from sklearn.metrics import confusion_matrix, classification_report, matthews_corrcoef, f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
from torchvision import models

#from PIL import Image
from tqdm import tqdm
import torch.nn as nn
from torchvision.models import resnet50
softmax = nn.Softmax(dim = 1)

In [30]:
# Set device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
!curl -L -o 'HatefulMemes.zip' 'https://drive.google.com/u/0/uc?id=1e--1ysAumCP6AtWpDvEFULLRLVMxCu5m&export=download&confirm=t'
shutil.unpack_archive("/content/HatefulMemes.zip", "/content/HatefulMemes")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 3430M  100 3430M    0     0   159M      0  0:00:21  0:00:21 --:--:--  177M


In [ ]:
lines = []
import json
with open('/content/HatefulMemes/archive/data/train.jsonl', 'r') as f:
    for line in f:
        line_val = json.loads(line)
        lines.append(line_val)
train_df = pd.DataFrame()
file_paths = []
labels = []
texts = []

for line in lines:
    file_paths.append("/content/HatefulMemes/archive/data/" + dict(line)["img"])
    labels.append(str(dict(line)["label"]))
    texts.append(str(dict(line)["text"]))

train_df["filepath"] = file_paths
train_df["label"] = labels
train_df["text"] = texts

lines = []

import json

with open('/content/HatefulMemes/archive/data/dev.jsonl', 'r') as f:
    for line in f:
        line_val = json.loads(line)
        lines.append(line_val)

valid_df = pd.DataFrame()
file_paths = []
labels = []
texts = []

for line in lines:
    file_paths.append("/content/HatefulMemes/archive/data/" + dict(line)["img"])
    labels.append(str(dict(line)["label"]))
    texts.append(str(dict(line)["text"]))

valid_df["filepath"] = file_paths
valid_df["label"] = labels
valid_df["text"] = texts

In [ ]:
nClasses = len(pd.unique(train_df['label']))
nClasses

# NLP - Approach

In [ ]:
!pip install -q transformers

In [ ]:
import transformers
from torch.utils.data import DataLoader, Dataset

# Define a custom dataset for our text data
class TextDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = self.data.iloc[index]['text']
        label = int(self.data.iloc[index]['label'])
        return text, label

In [ ]:
# Load the BERT tokenizer
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

# Define a collate function to prepare data for BERT model
def collate_fn(batch):
    texts, labels = zip(*batch)
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors='pt').to(device)
    labels = torch.tensor(labels).to(device)
    return inputs, labels

In [ ]:
# Create a dataloader for training set
train_dataset = TextDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

In [ ]:
# Create a dataloader for validation set
val_dataset = TextDataset(valid_df)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

In [ ]:
# Load the pre-trained BERT model and add a classification layer
model = transformers.BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=nClasses)
model.to(device)

In [ ]:
# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss().to(device)

In [ ]:
predictions_nlp = []
# Train the model
for epoch in range(1):
    model.train()
    for inputs, labels in tqdm(train_loader):
        optimizer.zero_grad()
        outputs = model(**inputs)[0]
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluate the model on validation set
    model.eval()
    val_loss = 0
    val_acc = 0
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader):
            outputs = model(**inputs)[0]
            val_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            predictions_nlp.append(softmax(outputs))
            val_acc += (predicted == labels).sum().item()
    val_loss /= len(val_loader.dataset)
    val_acc /= len(val_loader.dataset)

    print(f'Epoch {epoch+1}: Val Loss = {val_loss:.4f}, Val Acc = {val_acc:.4f}')

In [ ]:
train_df.head()